# NLP workshop

In [ ]:
!python -m spacy download es_core_news_md

In [22]:
# Importing required libraries for web scrapping

import time
import datetime

from bs4 import BeautifulSoup
from selenium import webdriver

from pymongo import MongoClient

In [23]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords

from wordcloud import WordCloud

import spacy
from spacy import displacy
import en_core_web_sm

import matplotlib.pyplot as plt
%matplotlib inline

In [52]:
import es_core_news_md
from nltk.tag import StanfordPOSTagger

import os

In [53]:
jar = "./pos-tagger/stanford-postagger/stanford-postagger-4.2.0.jar"
model = "./pos-tagger/stanford-postagger/models/spanish-ud.tagger"
os.environ["JAVAHOME"] = "/usr/bin/java"

In [24]:
# Creating a connection to MongoDB
client = MongoClient('localhost', 27017)
db = client['news-workshop']
collection = db['elespectador-workshop']

In [25]:
# Base URL of the site to be analyzed
SITE_URL = 'https://www.elespectador.com'

In [26]:
# Firefox web driver path
# Download the driver for you S.O. here: https://github.com/mozilla/geckodriver/releases
DRIVER_PATH = './geckodriver.exe'

In [42]:
# Creating a new firefox window
browser = webdriver.Firefox(executable_path = DRIVER_PATH)

C:\Users\BORISL~1\AppData\Local\Temp/ipykernel_2424/1910320475.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(executable_path = DRIVER_PATH)


In [43]:
def make_request(browser, relative_path):
    # Making the request and rendering the browser
    browser.get(SITE_URL + relative_path)
    
    # Simulating vertical scrolling for handling lazy load
    check_height = browser.execute_script('return document.body.scrollHeight;')
    while True:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(3)
        height = browser.execute_script('return document.body.scrollHeight;')
        if height == check_height: 
            break 
        check_height = height
    
    # Getting HTML content and passing it to BeautifulSoup for scraping analysis
    return BeautifulSoup(browser.page_source, 'html.parser')

In [44]:
categories = ['/archivo/politica/elecciones-colombia-2022/', '/archivo/tecnologia/', '/archivo/ambiente/', '/archivo/salud/', '/archivo/educacion/' ]

paginator = 1

news = []

In [48]:
for category in categories:
    # Getting HTML content for a particular news listing page
    wand = True
    
    while wand:
        page = "{}{}/".format(category, paginator)
        soup = make_request(browser, page)
    
        # Finding the section where news are contained 
        layout = soup.find(class_ = 'Layout-flexAds')
    
        # Getting blocks from layout
        blocks = layout.find_all(class_ = 'Container Block', recursive = True)
    
        # Finding and concatenating news cards
        cards = blocks[0].find_all(class_ = 'Card_rowCardLeft') + blocks[1].find_all(class_ = 'Card_rowCardLeft')
        
        for card in cards[:2]:
            date = card.find(class_ = 'Card-Datetime Datetime').get_text()
            
            x = datetime.datetime.now()- datetime.timedelta(weeks=2)
            months = ['feb.', 'mar.', 'apr.']
            partes = date.split(" ")
            if (partes[1]=='abr.'):
                partes[1] = 'apr.'
            elif(partes[1] not in months):
                partes[1] = 'apr.'
                partes[0] = datetime.datetime.now().day
                partes[2] = datetime.datetime.now().year
            fecha = "{}-{}-{}".format(partes[0], partes[1].replace(".",""), partes[2])
            date = datetime.datetime.strptime(fecha, '%d-%b-%Y')
            if date<x:
                wand = False
            
            # Getting HTML content for each news page
            r_path = card.find('h2', class_ = 'Card-Title').find('a')['href']
            soup = make_request(browser, r_path)
            
            author = ""
            paragraphs = soup.find_all(class_ = 'font--secondary')
            
            if soup.find(class_ = 'ACredit-Author').find('a'):
                author = soup.find(class_ = 'ACredit-Author').find('a').get_text()
            else:
                author = soup.find(class_ = 'ACredit-Author').get_text()
            
            categ = category.split("/")[2]
            
            news.append({
                'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
                'relative_path': r_path,
                'datetime' : date,
                'author' : author,
                'summary' : soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text(),
                'full_text' : ' '.join([p.get_text() for p in paragraphs]),
                'category' : categ,
            })
        
        paginator+=1
    
    paginator = 1
            

In [ ]:
news

In [49]:
# Storing extracted information for further analysis
collection.insert_many(news)

In [54]:
nlp = es_core_news_md.load()

In [ ]:
# Sentence tokenization
sent_tokenize(text)[:10]

In [ ]:
# Word tokenization
words = word_tokenize(text)

In [ ]:
words[:10]

In [ ]:
# Words distribution
frec_dist = FreqDist(words)

In [ ]:
# Printing the most common words
frec_dist.most_common(10)

In [ ]:
# Stop words
# Stop words are basically a set of commonly used words in any language, not just English.
# The reason why stop words are critical to many applications is that, if we remove the words that are very commonly used in a given language, we can focus on the important words instead.

stop_words = set(stopwords.words("english"))

In [ ]:
# Removing stop words
filtered_words = []

for w in words:
    if w.lower() not in stop_words:
        filtered_words.append(w)

In [ ]:
# Words distribution for text without stopwords
frec_dist_filtered = FreqDist(filtered_words)

In [ ]:
# Printing the most common words
frec_dist_filtered.most_common(10)

In [ ]:
# Plotting a bar char for frecuencies
n = 10

plt.figure(figsize = (15, 8))
plt.barh([ w[0] for w in frec_dist_filtered.most_common(n) ], [ w[1] for w in frec_dist_filtered.most_common(n) ])
plt.gca().invert_yaxis()

In [ ]:
# Creating the wordcloud
# Word size is directly related to frecuency
wordcloud = WordCloud(max_font_size = 50, max_words = 100, background_color = "white").generate(text)

plt.figure(figsize = (10, 10))
plt.imshow(wordcloud, interpolation = "bilinear")
plt.axis("off")
plt.show()

In [ ]:
# Creating the wordcloud
# Word size is directly related to frecuency
wordcloud = WordCloud(max_font_size = 50, max_words = 100, background_color = "white").generate(" ".join(filtered_words))

plt.figure(figsize = (10, 10))
plt.imshow(wordcloud, interpolation = "bilinear")
plt.axis("off")
plt.show()

In [ ]:
# Part of Speech (PoS)
# The meaning of each tag can be found here: https://www.guru99.com/pos-tagging-chunking-nltk.html
pos_tags = nltk.pos_tag(filtered_words)

In [ ]:
# Sorting tags
pos_tags.sort(key = lambda tup: tup[1])
pos_tags

In [ ]:
# Named Entity Recognition (NER)
document = nlp(text)
displacy.render(document, style = "ent")